In [ ]:
from imutils.video.filevideostream import FileVideoStream
from keras.models import load_model
from matplotlib import pyplot
from mtcnn.mtcnn import MTCNN
import numpy as np
from thundersvm import SVC
import argparse
import cv2

In [ ]:
# create the detector, using default weights
detector = MTCNN()
facenet_model = load_model('facenet_keras.h5')
thundersvm = SVC()
thundersvm.load_from_file('model.txt')

def detect_and_recognize(frame):
    # detect faces in the image    
    detections = detector.detect_faces(frame)
    
    for item in detections:
        x, y, width, height = item['box']
        
        face = frame[x:x+width, y:y+height]
        face = cv2.resize(face, (160, 160))
        face = face.astype('float32')
        
        # standardize pixel values across channels (global)
        mean, std = face.mean(), face.std()
        face = (face - mean) / std
        sample = np.expand_dims(face, axis=0)
        
        # make prediction to get embedding
        embedding = facenet_model.predict(sample)
        label = thundersvm.predict(embedding[0])
        
        (label_width, label_height), baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)
        
        frame = cv2.rectange(frame, (x, y), (x+width, y+height), color=(0, 0, 255) thickness=2)
        frame = cv2.rectange(frame, (x, y), (x+label_width, y+label_height), color=(0, 0, 255) thickness=-1)
        
        cv2.putText(image, label, (x, y-2), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (190, 190, 190), 2)
        
        cv2.imshow("output", frame)
        
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

In [ ]:
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True, help="path to the image")
ap.add_argument("-v", "--video", required=True, help="path to the video")
args = vars(ap.parse_args())

fvs = FileVideoStream(args["video"]).start()
time.sleep(1)

while True:
    frame = fvs.read()

    if frame is None:
        break
    
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    detect_and_recognize(frame)

fvs.stop()